## Implementing a Decision Tree

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

df= pd.read_csv('./day5_titanic/train.csv')
df.head(2)

In [ ]:
y=df['Survived']
X=df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']]

#### Although a decision tree can handle categorical features and missing data, using the same preprocessing for the sake of comparison with other implemented algorithms. Will compute again to check perforamnce without preprocessing

In [ ]:
X_numeric = X[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]
X_categorical = X[['Sex','Embarked']]
X_neither = X['Cabin']
X_onehot = pd.get_dummies(X_categorical,['Sex','Embarked']).astype(int)
X_cabin_nonna = X['Cabin'].apply(lambda x: 0 if pd.isna(x) else 1)
X_encoded0 = pd.merge(X_numeric, X_cabin_nonna,  left_index=True, right_index=True)
X_encoded1 = pd.merge(X_encoded0, X_onehot,  left_index=True, right_index=True)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded1, y, test_size=0.2, random_state=42, stratify=y
)
from sklearn.impute import SimpleImputer

# Imputer for numeric columns (e.g., Age)
imputer = SimpleImputer(strategy='median')  # or 'mean' if you prefer

# Fit on train, transform train
X_train['Age'] = imputer.fit_transform(X_train[['Age']])

# Transform test using same statistics
X_test['Age'] = imputer.transform(X_test[['Age']])

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### Performance on sklearns implementation
Hyperparameters: min examples per leaf, max tree depth, min impurity reduction on split
Start at X_train_scaled, X_test_scaled, y_train, y_test and apply decision tree

In [ ]:
# ----------------------------
# Import libraries
# ----------------------------
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# ----------------------------
# Create and train Decision Tree model
# ----------------------------
# You can tune parameters like max_depth, min_samples_split, etc.
dt_model = DecisionTreeClassifier(random_state=42,max_depth=3)

# Train the model
dt_model.fit(X_train_scaled, y_train)

# ----------------------------
# Make predictions
# ----------------------------
y_train_pred = dt_model.predict(X_train_scaled)
y_test_pred = dt_model.predict(X_test_scaled)

# ----------------------------
# Evaluate accuracy
# ----------------------------
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"Decision Tree Train Accuracy: {train_acc:.4f}")
print(f"Decision Tree Test Accuracy:  {test_acc:.4f}")


In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))  # adjust size as needed
tree.plot_tree(
    dt_model,                 # your trained Decision Tree
    feature_names=X_train.columns,  # column names
    class_names=['0','1'],    # labels for classes
    filled=True,              # color nodes by class
    rounded=True,             # rounded boxes
    fontsize=12
)
plt.show()


### Implementation from scratch

In [ ]:
def entropy(y):
    '''
    y is a pandas series. return float value of entropy
    '''
    eps = 1e-9
    n_samples = y.shape[0]
    
    y = np.asarray(y)
    cats, counts = np.unique(y, return_counts=True)
    pi =  (counts/ n_samples) + eps
    pi = pi / sum(pi)
    log_pi = np.log2(pi)
    entropy = np.dot(pi,log_pi)
    return -1 *  entropy
      

In [ ]:
import pandas as pd
y = pd.Series(['cat', 'cat', 'dog', 'dog', 'dog'])
print(entropy(y))  # Output: ~0.97095

In [ ]:
def information_gain(y, y_l, y_r):
    '''
    computes infromation gain when y is split into y_l and y_r
    '''
    y, y_l, y_r = map(np.asarray, (y, y_l, y_r))
    n, n_l, n_r = (arr.shape[0] for arr in (y,y_l,y_r))
    if n_l == 0 or n_r == 0:
        return 0
    
    term1 = entropy(y)
    term2 = (n_l)*entropy(y_l)/(n)
    term3 = (n_r)*entropy(y_r)/(n)
    return term1 - term2 - term3
    

In [ ]:
y = pd.Series(['cat', 'cat', 'dog', 'dog', 'dog'])
y_l = pd.Series(['cat', 'cat'])
y_r = pd.Series(['dog', 'dog', 'dog'])
print(information_gain(y, y_l, y_r))  # Output: ~0.97095

In [ ]:
y_l = pd.Series(['cat', 'cat','dog'])
y_r = pd.Series(['dog', 'dog'])
print(information_gain(y, y_l, y_r))  # Output: ~0.97095

In [ ]:
def get_most_common_y(y):
        y_unique, freq = np.unique(y, return_counts=True)
        return y_unique[ np.argmax(freq)]

get_most_common_y([1,2,1,1,4,4,4,4])

In [ ]:
class Node:
    def __init__(self, depth = 0):
        self.left = None
        self.right = None
        self.val = None
        
        self.depth = depth
        self.information_gain = None
        self.feature_index = None
        self.threshold = None

    

class DTree:
    def __init__(self, max_depth = 3, min_examples_leaf = 10):
        self.max_depth = max_depth
        self.min_examples_leaf = min_examples_leaf
        self.root = None

    def _get_most_common_y(self, y):
        y_unique, freq = np.unique(y, return_counts=True)
        return y_unique[np.argmax(freq)]

    def fit(self, X, y, depth=0):
        n_samples, n_features = X.shape
        
        if n_samples <= self.min_examples_leaf or len(np.unique(y))==1 or depth>=self.max_depth :
            leaf = Node(depth)
            leaf.val = self._get_most_common_y(y)
            return leaf
        else:
            
            max_inf_gain = -1
            feature_idx = None
            split_point = None
            best_splits = None
            for feature_number in range(n_features):
                x_iter = X[:,feature_number]
                x_iter_sorted = np.sort(x_iter)
                for idx, point in enumerate(x_iter_sorted[:-1]):
                    midpoint = np.mean([x_iter_sorted[idx],x_iter_sorted[idx+1]]) 
                    X_lower_indices =  X[:,feature_number]<midpoint
                    X_higher_indices = X[:,feature_number]>=midpoint
                    inf_gain = information_gain(y,y[X_lower_indices],y[X_higher_indices])
                    if inf_gain > max_inf_gain:
                        max_inf_gain = inf_gain
                        split_point = midpoint
                        feature_idx = feature_number
                        best_splits = (X_lower_indices, X_higher_indices)
            if max_inf_gain <=0 or len(y[X_lower_indices]) == 0 or len(y[X_higher_indices]) == 0: ## Should the equality be here, yes, if info gain is 0 there is no point in splitting 
            ### need the len checks above since previous features have turned the max_inf_gain to be positive 
                leaf = Node(depth)
                leaf.val = self._get_most_common_y(y)
                return leaf
            else:
                node = Node(depth)
                node.left = self.fit(X[X_lower_indices],y[X_lower_indices],depth+1)
                node.right = self.fit(X[X_higher_indices],y[X_higher_indices],depth+1)
                
                self.information_gain = max_inf_gain
                self.feature_index = feature_idx
                self.threshold = split_point
                return node
                


    def predict_one_example(self, x, node=None):
        '''
        x is a list of n_features * 1, why is node passed when we already have self. node is an object of a different class. 
        '''
        if node is None:
            node = self.root
        if node.val is not None: ## leaf node
            return node.val
        feature_idx = node.threshold
        thresh = node.feature_index
        print(x[feature_idx], thresh)
        if x[feature_idx] < thresh:
            return self.predict_one_example(x,node.left)
        else:
            return self.predict_one_example(x,node.right)
        
    def predict_all(self, X):
        '''
        y is a list of n_examples dimension
        '''
        y = [self.predict_one_example(x) for x in X]
        
    def train(self, X, y):
        """Public API for fitting"""
        self.root = self.fit(X, y, 0)     
                

#### Creating a new PR to test DTree class

In [ ]:
a = DTree()
a.train(X_train_scaled,y_train)

In [ ]:
y_train_pred = a.predict_one_example(X_train_scaled[0,:])
type(y_train_pred)

In [ ]:
X_train_scaled[0,:]

In [ ]:
X = np.array([[3,4,5,6,7,8],[1,2,3,4,5,6]])

def predict_per_row(row):
    return np.max(row)

def predict_all(X):
    '''
        y is a list of n_examples dimension
    '''
    y = [predict_per_row(x) for x in X]
    return (y)



print(predict_all(X))

#### Q1. How is a split evaluated in the code, we do a split, then what ?

### Random Forest

In [ ]:
# ----------------------------
# Import libraries
# ----------------------------
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# ----------------------------
# Create and train Random Forest model
# ----------------------------
rf_model = RandomForestClassifier(
    n_estimators=100,      # number of trees
    max_depth=3,           # depth of each tree
    random_state=42
)

# Train the model
rf_model.fit(X_train_scaled, y_train)

# ----------------------------
# Make predictions
# ----------------------------
y_train_pred = rf_model.predict(X_train_scaled)
y_test_pred = rf_model.predict(X_test_scaled)

# ----------------------------
# Evaluate accuracy
# ----------------------------
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"Random Forest Train Accuracy: {train_acc:.4f}")
print(f"Random Forest Test Accuracy:  {test_acc:.4f}")


### Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# Base estimator: shallow decision tree
base_dt = DecisionTreeClassifier(max_depth=1, random_state=42)

# Create AdaBoost model (new parameter name)
ada_model = AdaBoostClassifier(
    estimator=base_dt,       # <-- use 'estimator' instead of 'base_estimator'
    n_estimators=29,
    learning_rate=1,
    random_state=42
)

# Train the model
ada_model.fit(X_train_scaled, y_train)

# Predict
y_train_pred = ada_model.predict(X_train_scaled)
y_test_pred = ada_model.predict(X_test_scaled)

# Evaluate accuracy
from sklearn.metrics import accuracy_score
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"AdaBoost Train Accuracy: {train_acc:.4f}")
print(f"AdaBoost Test Accuracy:  {test_acc:.4f}")
